# Import and load Data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from wordcloud import WordCloud, STOPWORDS
from collections import OrderedDict
import json
from collections import defaultdict

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/train.csv")
test=pd.read_csv("../input/test.csv")

In [ ]:
data.head()

In [ ]:
data.info()

# Data Analysis #

### Revenue vs Homepage

In [ ]:
#we have a lot of null values for homepage
#Checking effect on homepage on revenue
data['has_homepage'] = 0
data.loc[data['homepage'].isnull() == False, 'has_homepage'] = 1
#Checking how homepage reflects on revenue

print('Number of null homepages in data = ',data[data.has_homepage==0]['id'].count(),'/',(data.id).count())
plt.figure(figsize=(6,12))
plt.scatter(data.has_homepage, data.revenue, alpha=0.2,
            s=50, cmap='viridis')
plt.xlabel('Does homepage exist?')
plt.ylabel('Total Revenue');
plt.xticks(np.arange(2), ('No','Yes'))
data=data.drop(['has_homepage'],axis =1)


### Revenue vs Tagline

In [ ]:
#Checking effect on tagline on revenue
data['has_tg'] = 0
data.loc[data['tagline'].isnull() == False, 'has_tg'] = 1

print('Number of null taglines in test = ',data[data.has_tg==0]['id'].count(),'/',(data.id).count())

plt.figure(figsize=(6,12))
plt.scatter(data.has_tg, data.revenue, alpha=0.2,
            s=50, cmap='viridis')
plt.xlabel('Does tagline exist?')
plt.ylabel('Total Revenue');

plt.xticks(np.arange(2), ('No','Yes'))
data=data.drop(['has_tg'],axis =1)

### Revenue vs Collection

In [ ]:
# Repeating same process for collections
data['has_collection'] = 0
data.loc[data['belongs_to_collection'].isnull() == False, 'has_collection'] = 1
print('Number of null collections in test = ',data[data.has_collection==0]['id'].count(),'/',(data.id).count())

plt.figure(figsize=(6,12))
plt.scatter(data.has_collection, data.revenue, alpha=0.2,
            s=50, cmap='viridis')
plt.xlabel('Does collection exist?')
plt.ylabel('Total Revenue');

plt.xticks(np.arange(2), ('No','Yes'))
data=data.drop(['has_collection'],axis =1)

### Correlation between features

In [ ]:
#Correlation between revenue, budget, popularity and runtime
col = ['revenue','budget','popularity','runtime']
plt.subplots(figsize=(10, 8))
corr = data[col].corr()
sns.heatmap(corr, xticklabels=col,yticklabels=col, linewidths=.5, cmap="Reds")

### Regression line of revenue using just budget

In [ ]:
sns.regplot(x="budget", y="revenue", data=data)

### Total Revenue by Day of Week

In [ ]:
#Relation of revenue on day of week when movie was released
def get_day_of_week(row):
    return pd.to_datetime(row.release_date).dayofweek
data['dow']= data.apply (lambda row: get_day_of_week(row), axis=1)
revenue_by_dow = data.groupby('dow')['revenue'].sum()
# number_of_movies = data.groupby('dow')['revenue'].count()
days = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
y_pos = np.arange(len(revenue_by_dow))
plt.figure(figsize=(15,10))
plt.bar(y_pos, revenue_by_dow, align='center', alpha=0.5)
plt.xticks(y_pos, days)
plt.xlabel('Day of Week')
plt.ylabel('Revenue')
plt.title('Total Revenue by Day of Week')
plt.show()

### Normalized Revenue by day of week

In [ ]:
plt.figure(figsize=(15,10))
number_of_movies = data.groupby('dow')['revenue'].count()
normalized_revenue = revenue_by_dow/number_of_movies
plt.bar(y_pos, normalized_revenue, align='center', alpha=0.5)
plt.xticks(y_pos, days)
plt.xlabel('Day of Week')
plt.ylabel('Revenue')
plt.title('Normalized Revenue by Day of Week')
plt.show()

### Revenue by Year

In [ ]:
#Relation of revenue on day of week when movie was released
def get_year(row):
    year = pd.to_datetime(row.release_date).year
    if(year>2019):
        return year-100
    return year
data['year']= data.apply (lambda row: get_year(row), axis=1)
revenue_by_year = data.groupby('year')['revenue'].sum().reset_index()
years = list(revenue_by_year.year)
y_pos = np.arange(revenue_by_year.revenue.count())
plt.figure(figsize=(15,10))
plt.bar(y_pos, revenue_by_year.revenue, align='center', alpha=0.5)
plt.xticks(y_pos, np.array(years)%100, rotation='vertical')
plt.xlabel('Year')
plt.ylabel('Revenue')
plt.title('Total Revenue by Year')
plt.show()
# revenue_by_year

### Revenue by genre

In [ ]:
mydict = defaultdict(int)
def populate_revenue_by_genre(row):
    if not pd.isnull(row.genres):
        my_genres = json.loads(row.genres.replace("\'", "\""))
        for genre in my_genres:
            mydict[genre["name"]]+=row.revenue
data.apply (lambda row: populate_revenue_by_genre(row), axis=1)
plt.figure(figsize=(25,10))

genres = list(mydict.keys())
revenue_by_genre = list(mydict.values())
y_pos = np.arange(len(genres))
plt.bar(y_pos, revenue_by_genre, align='center', alpha=0.5)
plt.xticks(y_pos, genres)
plt.xlabel('Genres')
plt.ylabel('Revenue')
plt.title('Total Revenue by Genre')
plt.show()

### Revenue by Language

In [ ]:
revenue_by_lang = data.groupby('original_language')['revenue'].sum().reset_index()
number_of_languages = revenue_by_lang.revenue.count()
y_pos = np.arange(number_of_languages)
plt.figure(figsize=(15,10))
plt.bar(y_pos, revenue_by_lang['revenue'], align='center', alpha=0.5)
plt.xticks(y_pos, revenue_by_lang['original_language'])
plt.xlabel('Languages')
plt.ylabel('Revenue')
plt.title('Total Revenue by Day of Week')
plt.show()

### Revenue by language (excluding en)

In [ ]:
revenue_by_lang = revenue_by_lang[revenue_by_lang.original_language!='en']
number_of_languages = revenue_by_lang.revenue.count()
y_pos = np.arange(number_of_languages)
plt.figure(figsize=(15,10))
plt.bar(y_pos, revenue_by_lang['revenue'], align='center', alpha=0.5)
plt.xticks(y_pos, revenue_by_lang['original_language'])
plt.xlabel('Languages')
plt.ylabel('Revenue')
plt.title('Total Revenue by Language')
plt.show()

In [ ]:
data[data.original_language=='zh']